In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mercari/sample_submission.csv
/kaggle/input/mercari/test.tsv
/kaggle/input/mercari/train.tsv


In [3]:
!pip install contractions

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


## Imports

In [4]:
import contractions
import regex as re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer, minmax_scale
from scipy.sparse import hstack
from sklearn.linear_model import Ridge
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize

/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [5]:
train  = pd.read_csv('/kaggle/input/mercari/train.tsv',sep='\t')
test  = pd.read_csv('/kaggle/input/mercari/test.tsv',sep='\t')

In [6]:
#dropping rows with price==0.0
train.drop(train[train['price']==0.0].index,inplace=True)

### train test split

In [7]:
#splitting training data for cross validation
train, cv = train_test_split(train, test_size = 0.20, random_state = 21)

### Filling NA

In [8]:
train.item_description.fillna('No description yet',inplace=True)
test.item_description.fillna('No description yet',inplace=True)
cv.item_description.fillna('No description yet',inplace=True)
train.category_name.fillna('others/others/others',inplace=True)
test.category_name.fillna('others/others/others',inplace=True)
cv.category_name.fillna('others/others/others',inplace=True)
train.brand_name.fillna('locals',inplace=True)
test.brand_name.fillna('locals',inplace=True)
cv.brand_name.fillna('locals',inplace=True)

In [9]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
598744,165099,Ivivva Fly Tech Short Sleeve Pink large,3,Kids/Girls (4+)/Tops & T-Shirts,locals,27.0,0,Hot pink Ivivva Fly Tech Short Sleeve top. Thi...
189329,612100,Ralph Lauren dresses,3,Kids/Girls 0-24 Mos/Dresses,locals,21.0,0,Worn a few times. In very good condition
978528,1425963,NIKE Therma fit Warm fleece boys jacket,3,Kids/Boys (4+)/Coats & Jackets,Nike,8.0,1,Size 7. In good condition but it has a name wr...
921292,1291818,"NYX EYEBROW GEL,ANGLED BRUSH",1,Beauty/Makeup/Eyes,NYX,12.0,1,"Both brand new,sealed,in box,free fast shippin..."
682031,1392571,Scrub to,2,Women/Athletic Apparel/Shirts & Tops,locals,9.0,0,No description yet


### Preprocessing

In [10]:
stop_words = set(stopwords.words('english'))-{"no","nor","not"}#removng not nor no from stop words

In [11]:
# #removng not nor no
# stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
#             "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
#             'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
#             'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
#             'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
#             'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
#             'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
#             'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
#             'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
#             'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
#             's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
#             've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
#             "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
#             "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
#             'won', "won't", 'wouldn', "wouldn't"]

In [12]:
#Stemming: Stemming is the process of producing morphological variants of a root/base word

ps = PorterStemmer()
def text_prep(val):
    val = contractions.fix(val)
    val = val.replace('\\r',' ')
    val = val.replace('\\"',' ')
    val = val.replace('\\n',' ')
    val = re.sub('[^A-Za-z0-9]+',' ',val)
    val = ' '.join(ps.stem(v) for v in val.split() if v.lower() not in stop_words)
    val = val.lower().strip()
    return val

In [13]:
#preprocessing item_description
train['proc_item_description'] = train.item_description.apply(text_prep)
test['proc_item_description'] = test.item_description.apply(text_prep)
cv['proc_item_description'] = cv.item_description.apply(text_prep)
#preprocessing name
train['proc_name'] = train.name.apply(text_prep)
test['proc_name'] = test.name.apply(text_prep)
cv['proc_name'] = cv.name.apply(text_prep)

In [14]:
#concatenating variuos text fields into one to reduce the dimensionality of text fields
train["text"]=train["proc_item_description"]+" "+train["proc_name"]+' '+train["category_name"]
#some names also contains brand names
train['newname']=train['proc_name']+' '+train['brand_name']

In [15]:
test["text"]=test["proc_item_description"]+" "+test["proc_name"]+' '+test["category_name"]
test['newname']=test['proc_name']+' '+test['brand_name']

In [16]:
cv["text"]=cv["proc_item_description"]+" "+cv["proc_name"]+' '+cv["category_name"]
cv['newname']=cv['proc_name']+' '+cv['brand_name']

In [17]:
train.head(5)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,proc_item_description,proc_name,text,newname
598744,165099,Ivivva Fly Tech Short Sleeve Pink large,3,Kids/Girls (4+)/Tops & T-Shirts,locals,27.0,0,Hot pink Ivivva Fly Tech Short Sleeve top. Thi...,hot pink ivivva fli tech short sleev top look ...,ivivva fli tech short sleev pink larg,hot pink ivivva fli tech short sleev top look ...,ivivva fli tech short sleev pink larg locals
189329,612100,Ralph Lauren dresses,3,Kids/Girls 0-24 Mos/Dresses,locals,21.0,0,Worn a few times. In very good condition,worn time good condit,ralph lauren dress,worn time good condit ralph lauren dress Kids/...,ralph lauren dress locals
978528,1425963,NIKE Therma fit Warm fleece boys jacket,3,Kids/Boys (4+)/Coats & Jackets,Nike,8.0,1,Size 7. In good condition but it has a name wr...,size 7 good condit name written inner shown la...,nike therma fit warm fleec boy jacket,size 7 good condit name written inner shown la...,nike therma fit warm fleec boy jacket Nike
921292,1291818,"NYX EYEBROW GEL,ANGLED BRUSH",1,Beauty/Makeup/Eyes,NYX,12.0,1,"Both brand new,sealed,in box,free fast shippin...",brand new seal box free fast ship free appreci...,nyx eyebrow gel angl brush,brand new seal box free fast ship free appreci...,nyx eyebrow gel angl brush NYX
682031,1392571,Scrub to,2,Women/Athletic Apparel/Shirts & Tops,locals,9.0,0,No description yet,no descript yet,scrub,no descript yet scrub Women/Athletic Apparel/S...,scrub locals


### Word to vec

### Convert a collection of raw documents to a matrix of TF-IDF features.

In [18]:
vectorizer = TfidfVectorizer()
vectorizer.fit(train['text'].values)
train_text = vectorizer.transform(train['text'].values)
test_text = vectorizer.transform(test['text'].values)
cv_text = vectorizer.transform(cv['text'].values)

In [19]:
vectorizer = CountVectorizer()
vectorizer.fit(train['newname'].values)
train_newname = vectorizer.transform(train['newname'].values)
test_newname = vectorizer.transform(test['newname'].values)
cv_newname = vectorizer.transform(cv['newname'].values)

In [20]:
train.shape

(829736, 12)

### Encoding

#### convirting catogorical variable into dummy variable(one hot encoding) and into compressed sparse row matrix

In [21]:
 train_item_cond = csr_matrix(pd.get_dummies(train[['item_condition_id','shipping']],
                                          sparse=True).values)

In [22]:
test_item_cond = csr_matrix(pd.get_dummies(test[['item_condition_id','shipping']],
                                          sparse=True).values)

In [23]:
cv_item_cond = csr_matrix(pd.get_dummies(cv[['item_condition_id','shipping']],
                                          sparse=True).values)

### Preparing Features

In [24]:
#stacking features for training
X_train_data = hstack((train_newname,train_text,train_item_cond))

In [25]:
#taking log(price)
Y_train_data = np.log(train['price'].values+1)

In [26]:
Y_cv_data = np.log(cv['price'].values+1)

In [27]:
#stacking features for testing
X_test_data = hstack((test_newname,test_text,test_item_cond))

In [28]:
#stacking features for cross validation
X_cv_data = hstack((cv_newname,cv_text,cv_item_cond))

In [29]:
#defining Root Mean Squared Logarithmic Error function
def rmse(a,p):
    s = np.sqrt(np.sum(np.square(np.log(p+1)-np.log(a+1)))/p.shape[0])
    return s

### Model: LightGBM

In [31]:
#number of boosted trees to fit=6000, maximum depth of tree=50,
#number of parallel threads for LightGBM= No limit(use all processor)
lgbm_params_1 = {'n_estimators':6000,
                 'num_leaves':80,
                 'max_depth':50
                }

In [32]:
import lightgbm as lgb

In [33]:
#Construct a gradient boosting model
model = lgb.LGBMRegressor(**lgbm_params_1)

In [34]:
#Build a gradient boosting model from the training set
model.fit(X_train_data, Y_train_data)

LGBMRegressor(max_depth=50, n_estimators=6000, num_leaves=80)

In [35]:
#get the predicted value for each sample(cross validation)
pred = model.predict(X_cv_data)

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [36]:
#find error for cv
rmse(cv.price,np.expm1(pred))

0.4305860869750612

In [37]:
#get the predicted value for test data
Y_pred =  model.predict(X_test_data)
#create submission file
sub = {"id":test.id.values,"price":np.expm1(Y_pred)}
submission = pd.DataFrame(sub)
submission.to_csv("submission10.csv",index=False,header=True)